# Preprocessing Data
### Notebook where I combine all of the datasets together (from 2 different websites) and perform data cleaning before modeling

In [2]:
import pandas as pd
import numpy as np
from functools import reduce
import unidecode
import fuzzymatcher
from fuzzymatcher import link_table, fuzzy_left_join

In [3]:
df_standard = pd.read_csv('Top_5_Leagues_Standard.csv')
df_shooting = pd.read_csv('Top_5_Leagues_Shooting.csv')
df_passing = pd.read_csv('Top_5_Leagues_Passing.csv')
df_pass_type = pd.read_csv('Top_5_Leagues_Pass_Types.csv')
df_shot_creation = pd.read_csv('Top_5_Leagues_Shot_Creation.csv')
df_defense = pd.read_csv('Top_5_Leagues_Defense.csv')
df_possession = pd.read_csv('Top_5_Leagues_Possession.csv')
df_misc = pd.read_csv('Top_5_Leagues_Misc.csv')

In [4]:
# Clean Player Name Column
def clean_names(df):
    for index, row in df.iterrows():
        end = row['Player'].find('\\')
        df.loc[index, 'Player'] = row['Player'][0:end]
    
    return df

# Clean datafreame
def clean_df(df):
    df.columns = df.iloc[0]

    # Remove blank first row
    df.drop(index=0, axis=0, inplace=True)
    df.drop(index=1, axis=0, inplace=True)
    df = df.reset_index(drop=True)
    df = clean_names(df)
    df = df.apply(pd.to_numeric, errors='ignore')
    
    return df

In [5]:
df_standard = clean_df(df_standard)
df_shooting = clean_df(df_shooting)
df_passing = clean_df(df_passing)
df_pass_type = clean_df(df_pass_type)
df_shot_creation = clean_df(df_shot_creation)
df_defense = clean_df(df_defense)
df_possession = clean_df(df_possession)
df_misc = clean_df(df_misc)

In [6]:
df_standard

,Player,Pos,Squad,Comp,Age,MP,Min,Gls,Ast,G+A,xG,xA
0,Ismael Aaneba,FWDF,Strasbourg,fr Ligue 1,21,2,11,0.00,0.00,0.00,0.00,0.00
1,Patrick van Aanholt,DF,Crystal Palace,eng Premier League,29,22,1777,0.00,0.05,0.05,0.06,0.04
2,Issah Abbas,DFFW,Mainz 05,de Bundesliga,21,2,18,0.00,0.00,0.00,0.00,0.00
3,Yunis Abdelhamid,DF,Reims,fr Ligue 1,32,33,2889,0.09,0.00,0.09,0.05,0.01
4,Sabit Abdulai,MF,Getafe,es La Liga,21,3,60,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
2817,Steven Zuber,DFMF,Eint Frankfurt,de Bundesliga,28,20,585,0.00,0.46,0.46,0.12,0.29
2818,Martín Zubimendi,MF,Real Sociedad,es La Liga,21,31,1882,0.00,0.00,0.00,0.03,0.03
2819,Martin Ødegaard,MF,Arsenal,eng Premier League,21,14,866,0.10,0.21,0.31,0.11,0.22
2820,Martin Ødegaard,MFFW,Real Madrid,es La Liga,21,7,234,0.00,0.00,0.00,0.04,0.15


In [7]:
df_shooting

,Player,Pos,Squad,Comp,Age,90s,SoT%,npxG/Sh,G-xG
0,Ismael Aaneba,FWDF,Strasbourg,fr Ligue 1,21,0.1,NaN,NaN,0.00
1,Patrick van Aanholt,DF,Crystal Palace,eng Premier League,29,19.7,33.3,0.07,-0.06
2,Issah Abbas,DFFW,Mainz 05,de Bundesliga,21,0.2,NaN,NaN,0.00
3,Yunis Abdelhamid,DF,Reims,fr Ligue 1,32,32.1,33.3,0.11,0.04
4,Sabit Abdulai,MF,Getafe,es La Liga,21,0.7,NaN,NaN,0.00
...,...,...,...,...,...,...,...,...,...
2817,Steven Zuber,DFMF,Eint Frankfurt,de Bundesliga,28,6.5,20.0,0.08,-0.12
2818,Martín Zubimendi,MF,Real Sociedad,es La Liga,21,20.9,14.3,0.08,-0.03
2819,Martin Ødegaard,MF,Arsenal,eng Premier League,21,9.6,20.0,0.07,-0.01
2820,Martin Ødegaard,MFFW,Real Madrid,es La Liga,21,2.6,0.0,0.04,-0.04


In [8]:
df_passing

,Player,Pos,Squad,Comp,Age,90s,Cmp%,Prog
0,Ismael Aaneba,FWDF,Strasbourg,fr Ligue 1,21,0.1,100.0,0.00
1,Patrick van Aanholt,DF,Crystal Palace,eng Premier League,29,19.7,78.9,5.69
2,Issah Abbas,DFFW,Mainz 05,de Bundesliga,21,0.2,86.7,5.00
3,Yunis Abdelhamid,DF,Reims,fr Ligue 1,32,32.1,87.8,2.83
4,Sabit Abdulai,MF,Getafe,es La Liga,21,0.7,58.3,1.43
...,...,...,...,...,...,...,...,...
2817,Steven Zuber,DFMF,Eint Frankfurt,de Bundesliga,28,6.5,76.0,4.62
2818,Martín Zubimendi,MF,Real Sociedad,es La Liga,21,20.9,89.3,4.59
2819,Martin Ødegaard,MF,Arsenal,eng Premier League,21,9.6,86.4,5.00
2820,Martin Ødegaard,MFFW,Real Madrid,es La Liga,21,2.6,84.0,3.46


In [9]:
df_pass_type

,Player,Pos,Squad,Comp,Age,90s,TB,Press,Sw,Crs
0,Ismael Aaneba,FWDF,Strasbourg,fr Ligue 1,21,0.1,0.00,0.00,10.00,0.00
1,Patrick van Aanholt,DF,Crystal Palace,eng Premier League,29,19.7,0.10,5.69,1.02,1.73
2,Issah Abbas,DFFW,Mainz 05,de Bundesliga,21,0.2,0.00,0.00,0.00,0.00
3,Yunis Abdelhamid,DF,Reims,fr Ligue 1,32,32.1,0.00,5.23,1.21,0.00
4,Sabit Abdulai,MF,Getafe,es La Liga,21,0.7,0.00,10.00,1.43,1.43
...,...,...,...,...,...,...,...,...,...,...
2817,Steven Zuber,DFMF,Eint Frankfurt,de Bundesliga,28,6.5,0.15,10.60,1.23,2.77
2818,Martín Zubimendi,MF,Real Sociedad,es La Liga,21,20.9,0.10,8.47,0.81,0.10
2819,Martin Ødegaard,MF,Arsenal,eng Premier League,21,9.6,0.31,7.50,1.15,1.25
2820,Martin Ødegaard,MFFW,Real Madrid,es La Liga,21,2.6,0.00,13.50,0.77,0.77


In [10]:
# Rows with duplicate player names
duplicate_names = df_standard[df_standard.duplicated('Player')].Player
# Rows with duplicate player names and age
duplicate_names_age = df_standard[df_standard.duplicated(subset=['Player', 'Age'])].Player
duplicate_player_names_only = duplicate_names[~duplicate_names.isin(duplicate_names_age)]
# Print rows with duplicate player names only
df_standard[df_standard.Player.isin(duplicate_player_names_only)]

,Player,Pos,Squad,Comp,Age,MP,Min,Gls,Ast,G+A,xG,xA
961,Raúl García,DFMF,Valladolid,es La Liga,31,8,504,0.18,0.00,0.18,0.02,0.04
962,Raúl García,FWMF,Athletic Club,es La Liga,34,34,1743,0.26,0.05,0.31,0.33,0.07
1249,Jota,FW,Valladolid,es La Liga,21,17,668,0.14,0.00,0.13,NaN,NaN
1250,Jota,MF,Alavés,es La Liga,29,23,1398,0.00,0.13,0.13,0.06,0.12
1606,Marcelo,DF,Real Madrid,es La Liga,32,16,1050,0.00,0.17,0.17,0.03,0.10
1607,Marcelo,DF,Lyon,fr Ligue 1,33,34,2907,0.09,0.00,0.09,0.04,0.02
1638,Javi Martínez,MFFW,Osasuna,es La Liga,20,13,460,0.20,0.20,0.39,0.12,0.10
1639,Javi Martínez,DFMF,Bayern Munich,de Bundesliga,31,19,466,0.00,0.00,0.00,0.06,0.04
1851,Nacho,DF,Real Madrid,es La Liga,30,24,2000,0.05,0.00,0.05,0.07,0.02
1852,Nacho,DF,Valladolid,es La Liga,31,23,1813,0.00,0.05,0.05,0.01,0.08


In [11]:
# Print rows with duplicate player names and age
df_standard[df_standard.Player.isin(duplicate_names_age)]

,Player,Pos,Squad,Comp,Age,MP,Min,Gls,Ast,G+A,xG,xA
35,Jean-Eudes Aholou,MF,Strasbourg,fr Ligue 1,26,27,2146,0.08,0.08,0.17,0.10,0.05
36,Jean-Eudes Aholou,MF,Monaco,fr Ligue 1,26,2,36,0.00,0.00,0.00,0.25,0.00
39,Rayan Aït Nouri,DF,Angers,fr Ligue 1,19,3,187,0.00,0.48,0.48,0.05,0.19
40,Rayan Aït Nouri,DF,Wolves,eng Premier League,19,21,1404,0.06,0.06,0.13,0.04,0.06
65,Carles Aleñá,MF,Barcelona,es La Liga,22,2,48,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
2778,Okay Yokuşlu,MF,Celta Vigo,es La Liga,26,12,462,0.00,0.00,0.00,0.08,0.06
2810,Joshua Zirkzee,FW,Bayern Munich,de Bundesliga,19,3,92,0.00,0.00,0.00,0.40,0.20
2811,Joshua Zirkzee,FWMF,Parma,it Serie A,19,4,113,0.00,0.00,0.00,0.08,0.15
2819,Martin Ødegaard,MF,Arsenal,eng Premier League,21,14,866,0.10,0.21,0.31,0.11,0.22


#### Since the duplicates with names only are players that happen to have the exact same name, we won't change those duplicates.
#### However, the duplicates with both names and age are all players that were transfered to a different team mid-season, so we will get rid of the duplicate entry with the lesser number of matches played

In [12]:
# Function to remove duplicates of players who transfered mid season
def remove_duplicates(df):
    df = df.sort_values('Min', ascending=False).drop_duplicates(subset=['Player', 'Age'], keep='first').reset_index(drop=True)
    return df

In [13]:
# Combine all the dataframes
data_frames = [df_standard, df_shooting, df_passing, df_pass_type, df_shot_creation, df_defense, df_possession, df_misc]
combined_df = reduce(lambda  left,right: pd.merge(left,right,on=['Player', 'Pos', 'Squad', 'Comp', 'Age'],
                                            how='outer'), data_frames)
combined_df = remove_duplicates(combined_df)
combined_df.drop(labels=['90s', '90s_x', '90s_y'], axis=1, inplace=True)

In [14]:
combined_df

,Player,Pos,Squad,Comp,Age,MP,Min,Gls,Ast,G+A,...,Press_y,Sh,Int,Clr,Err,Succ,PrgDist,Mis,Recov,Won%
0,Walter Benítez,GK,Nice,fr Ligue 1,27,38,3420,0.00,0.0,0.00,...,0.08,0.0,0.00,0.00,0.05,0.00,33.5,0.03,3.95,NaN
1,Bruno Ecuele Manga,DF,Dijon,fr Ligue 1,32,38,3420,0.05,0.0,0.05,...,5.92,1.0,2.16,5.50,0.05,0.26,89.5,0.16,9.16,69.7
2,Álex Remiro,GK,Real Sociedad,es La Liga,25,38,3420,0.00,0.0,0.00,...,0.08,0.0,0.00,0.05,0.00,0.03,40.1,0.00,4.76,NaN
3,Mike Maignan,GK,Lille,fr Ligue 1,24,38,3420,0.00,0.0,0.00,...,0.05,0.0,0.00,0.05,0.05,0.03,36.4,0.05,5.13,NaN
4,Aaron Ramsdale,GK,Sheffield Utd,eng Premier League,22,38,3420,0.00,0.0,0.00,...,0.21,0.0,0.00,0.03,0.08,0.05,50.3,0.03,4.82,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2699,Jayson Molumby,MF,Brighton,eng Premier League,20,1,1,0.00,0.0,0.00,...,1.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,1.00,NaN
2700,Ramón Folch,MF,Elche,es La Liga,30,1,1,0.00,0.0,0.00,...,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,NaN
2701,Marten Winkler,FW,Hertha BSC,de Bundesliga,17,1,1,0.00,0.0,0.00,...,2.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0
2702,Sergio Cubero Ezcurra,MF,Eibar,es La Liga,20,1,1,0.00,0.0,0.00,...,1.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,NaN


In [15]:
combined_df.columns

Index(['Player', 'Pos', 'Squad', 'Comp', 'Age', 'MP', 'Min', 'Gls', 'Ast',
       'G+A', 'xG', 'xA', 'SoT%', 'npxG/Sh', 'G-xG', 'Cmp%', 'Prog', 'TB',
       'Press_x', 'Sw', 'Crs', 'SCA', 'Drib', 'Def', 'Tkl', 'Press_y', 'Sh',
       'Int', 'Clr', 'Err', 'Succ', 'PrgDist', 'Mis', 'Recov', 'Won%'],
      dtype='object', name=0)

In [16]:
# Rename column values
combined_df.rename(columns={"Pos": "Position", "Squad": "Team", "Gls":"Goals",
                            "Cmp%":"Pass Cmp%", "Prog":"Prog Passes", "TB":"Through Balls",
                            "Press_x":"Passes Under Pressure", "Sw":"Switch Passes", "Crs":"Crosses",
                            "SCA":"Shot Creating Actions", "Drib":"Dribbles to Shot", "Def":"Def Action to Shot", "Tkl":"Tackles",
                            "Press_y":"Press", "Sh":"Shots Blocked", "Int":"Interceptions", "Clr":"Clearances",
                            "Err":"Errors", "Succ":"Dribbles Successful", "PrgDist":"Dribble Distance", "Mis":"Mistakes",
                            "Recov":"Ball Recoveries", "Won%":"Air Duels Won%",}, inplace=True)
combined_df.columns

Index(['Player', 'Position', 'Team', 'Comp', 'Age', 'MP', 'Min', 'Goals',
       'Ast', 'G+A', 'xG', 'xA', 'SoT%', 'npxG/Sh', 'G-xG', 'Pass Cmp%',
       'Prog Passes', 'Through Balls', 'Passes Under Pressure',
       'Switch Passes', 'Crosses', 'Shot Creating Actions', 'Dribbles to Shot',
       'Def Action to Shot', 'Tackles', 'Press', 'Shots Blocked',
       'Interceptions', 'Clearances', 'Errors', 'Dribbles Successful',
       'Dribble Distance', 'Mistakes', 'Ball Recoveries', 'Air Duels Won%'],
      dtype='object', name=0)

In [17]:
# Drop goalkeepers from dataset
combined_df.drop(combined_df[combined_df.Position == 'GK'].index, inplace=True)
combined_df.drop(combined_df[combined_df.Position == 'GKMF'].index, inplace=True)

# Drop any players that haven't played 90 minutes
combined_df.drop(combined_df[combined_df.Min < 90].index, inplace=True)
combined_df.reset_index(drop=True, inplace=True)
combined_df

,Player,Position,Team,Comp,Age,MP,Min,Goals,Ast,G+A,...,Press,Shots Blocked,Interceptions,Clearances,Errors,Dribbles Successful,Dribble Distance,Mistakes,Ball Recoveries,Air Duels Won%
0,Bruno Ecuele Manga,DF,Dijon,fr Ligue 1,32,38,3420,0.05,0.00,0.05,...,5.92,1.00,2.16,5.50,0.05,0.26,89.5,0.16,9.16,69.7
1,Pierre Højbjerg,MF,Tottenham,eng Premier League,24,38,3420,0.05,0.11,0.16,...,21.20,0.26,1.29,2.16,0.00,0.66,114.1,0.74,11.30,56.5
2,James Ward-Prowse,MF,Southampton,eng Premier League,25,38,3420,0.21,0.18,0.39,...,16.60,0.13,1.16,2.00,0.00,0.39,91.4,0.42,12.70,43.4
3,Tomáš Souček,MF,West Ham,eng Premier League,25,38,3419,0.26,0.03,0.29,...,15.90,0.55,1.45,3.82,0.03,0.42,53.7,0.89,12.20,63.0
4,Dylan Bronn,DF,Metz,fr Ligue 1,25,38,3412,0.05,0.00,0.05,...,10.90,0.42,1.00,4.30,0.08,0.42,117.3,0.13,8.42,57.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2198,Kayne Ramsey,DF,Southampton,eng Premier League,19,1,90,0.00,0.00,0.00,...,8.00,1.00,1.00,4.00,0.00,0.00,12.0,0.00,6.00,50.0
2199,Yoann Etienne,DF,Lorient,fr Ligue 1,23,1,90,0.00,0.00,0.00,...,21.00,0.00,2.00,0.00,0.00,0.00,48.0,2.00,10.00,25.0
2200,Ahmed Hegazi,DF,West Brom,eng Premier League,29,1,90,0.00,0.00,0.00,...,10.00,0.00,0.00,11.00,0.00,0.00,62.0,1.00,21.00,72.7
2201,Márk Kosznovszky,MF,Parma,it Serie A,18,1,90,0.00,0.00,0.00,...,16.00,0.00,3.00,0.00,0.00,0.00,69.0,1.00,8.00,NaN


In [18]:
# Get rid of null values (any percent columns replace null with mean, any other columns replace null with 0)

# Function to find percent columns
def find_percent_cols(df):
    percent_cols = []
    for name in df.columns:
        if '%' in name:
            percent_cols.append(name)
            
    return percent_cols
        
# Function to get rid of null values
def remove_null(df):
    percent_cols = find_percent_cols(df)
    
    for col in percent_cols:
        df[col].fillna(round(df[col].mean(), 1), inplace=True)
        
    df.fillna(0, inplace=True)
        
    return df

In [19]:
combined_df = remove_null(combined_df)
combined_df

,Player,Position,Team,Comp,Age,MP,Min,Goals,Ast,G+A,...,Press,Shots Blocked,Interceptions,Clearances,Errors,Dribbles Successful,Dribble Distance,Mistakes,Ball Recoveries,Air Duels Won%
0,Bruno Ecuele Manga,DF,Dijon,fr Ligue 1,32,38,3420,0.05,0.00,0.05,...,5.92,1.00,2.16,5.50,0.05,0.26,89.5,0.16,9.16,69.7
1,Pierre Højbjerg,MF,Tottenham,eng Premier League,24,38,3420,0.05,0.11,0.16,...,21.20,0.26,1.29,2.16,0.00,0.66,114.1,0.74,11.30,56.5
2,James Ward-Prowse,MF,Southampton,eng Premier League,25,38,3420,0.21,0.18,0.39,...,16.60,0.13,1.16,2.00,0.00,0.39,91.4,0.42,12.70,43.4
3,Tomáš Souček,MF,West Ham,eng Premier League,25,38,3419,0.26,0.03,0.29,...,15.90,0.55,1.45,3.82,0.03,0.42,53.7,0.89,12.20,63.0
4,Dylan Bronn,DF,Metz,fr Ligue 1,25,38,3412,0.05,0.00,0.05,...,10.90,0.42,1.00,4.30,0.08,0.42,117.3,0.13,8.42,57.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2198,Kayne Ramsey,DF,Southampton,eng Premier League,19,1,90,0.00,0.00,0.00,...,8.00,1.00,1.00,4.00,0.00,0.00,12.0,0.00,6.00,50.0
2199,Yoann Etienne,DF,Lorient,fr Ligue 1,23,1,90,0.00,0.00,0.00,...,21.00,0.00,2.00,0.00,0.00,0.00,48.0,2.00,10.00,25.0
2200,Ahmed Hegazi,DF,West Brom,eng Premier League,29,1,90,0.00,0.00,0.00,...,10.00,0.00,0.00,11.00,0.00,0.00,62.0,1.00,21.00,72.7
2201,Márk Kosznovszky,MF,Parma,it Serie A,18,1,90,0.00,0.00,0.00,...,16.00,0.00,3.00,0.00,0.00,0.00,69.0,1.00,8.00,46.4


In [20]:
combined_df.isna().sum()

0
Player                   0
Position                 0
Team                     0
Comp                     0
Age                      0
MP                       0
Min                      0
Goals                    0
Ast                      0
G+A                      0
xG                       0
xA                       0
SoT%                     0
npxG/Sh                  0
G-xG                     0
Pass Cmp%                0
Prog Passes              0
Through Balls            0
Passes Under Pressure    0
Switch Passes            0
Crosses                  0
Shot Creating Actions    0
Dribbles to Shot         0
Def Action to Shot       0
Tackles                  0
Press                    0
Shots Blocked            0
Interceptions            0
Clearances               0
Errors                   0
Dribbles Successful      0
Dribble Distance         0
Mistakes                 0
Ball Recoveries          0
Air Duels Won%           0
dtype: int64

#### Now we will combine the stats dataset from FB-Reference to the transfer market dataset

In [21]:
# Load transfer market dataset
tm_df = pd.read_csv('top5_league_market_values.csv')
tm_df.drop(labels=tm_df.columns[0], axis=1, inplace=True)
tm_df.rename(columns={"Players": "Player"}, inplace=True)

# Get rid of accents and special letters/characters, also 
tm_df_player_list = []
combined_df_player_list = []
tm_df_team_list = []
combined_df_team_list = []

for index, row in tm_df.iterrows():
    player_name = unidecode.unidecode(row['Player'])
    team_name = unidecode.unidecode(row['Team'])
    tm_df_player_list.append(player_name)
    tm_df_team_list.append(team_name)
for index, row in combined_df.iterrows():
    player_name = unidecode.unidecode(row['Player'])
    team_name = unidecode.unidecode(row['Team']).lower()
    combined_df_player_list.append(player_name)
    combined_df_team_list.append(team_name)
tm_df.Player = tm_df_player_list
combined_df.Player = combined_df_player_list
tm_df.Team = tm_df_team_list
combined_df.Team = combined_df_team_list

tm_df

,Player,Values,Team
0,Ederson,55000000.0,manchester city
1,Zack Steffen,6600000.0,manchester city
2,Scott Carson,330000.0,manchester city
3,Ruben Dias,82500000.0,manchester city
4,Aymeric Laporte,49500000.0,manchester city
...,...,...,...
3053,Moussa Konate,2750000.0,dijon fco
3054,Aboubakar Kamara,2200000.0,dijon fco
3055,Aurelien Scheidler,1100000.0,dijon fco
3056,Rayan Philippe,770000.0,dijon fco


In [23]:
# Change the team names so they match in both datasets
replace_dict = {'deportivo alaves':'alaves', 'sco angers':'angers', 'arminia bielefeld':'arminia',
 'arsenal fc':'arsenal', 'atalanta bc':'atalanta', 'athletic bilbao':'athletic club', 'atletico de madrid':'atletico madrid',
 'fc augsburg':'augsburg', 'fc barcelona':'barcelona', 'benevento calcio':'benevento',
 'real betis balompie':'betis', 'bologna fc 1909':'bologna', 'fc girondins bordeaux':'bordeaux',
 'stade brest 29':'brest', 'brighton amp hove albion':'brighton', 'burnley fc':'burnley',
 'cadiz cf':'cadiz', 'cagliari calcio':'cagliari', 'celta de vigo':'celta vigo',
 'chelsea fc':'chelsea', 'fc crotone':'crotone', 'dijon fco':'dijon',
 'borussia dortmund':'dortmund', 'sd eibar':'eibar', 'eintracht frankfurt':'eint frankfurt',
 'elche cf':'elche', 'everton fc':'everton', 'acf fiorentina':'fiorentina',
 'sc freiburg':'freiburg', 'fulham fc':'fulham', 'genoa cfc':'genoa',
 'getafe cf':'getafe', 'granada cf':'granada', 'tsg 1899 hoffenheim':'hoffenheim',
 'sd huesca':'huesca', 'inter milan':'inter', 'juventus fc':'juventus',
 '1 fc koln':'koln', 'ss lazio':'lazio', 'rc lens':'lens',
 'levante ud':'levante', 'bayer 04 leverkusen':'leverkusen', 'losc lille':'lille',
 'liverpool fc':'liverpool', 'fc lorient':'lorient', 'olympique lyon':'lyon',
 'borussia monchengladbach':"m'gladbach", '1 fsv mainz 05':'mainz 05', 'manchester united':'manchester utd',
 'olympique marseille':'marseille', 'fc metz':'metz', 'ac milan':'milan', 'as monaco':'monaco',
 'montpellier hsc':'montpellier', 'fc nantes':'nantes', 'ssc napoli':'napoli',
 'newcastle united':'newcastle utd', 'ogc nice':'nice', 'nimes olympiqu':'nimes', 'ca osasuna':'osasuna',
 'paris saint germain':'paris s-g', 'parma calcio 1913':'parma', 'stade reims':'reims', 'stade rennais fc':'rennes',
 'as roma':'roma', 'as saint etienne':'saint-etienne', 'uc sampdoria':'sampdoria', 'us sassuolo':'sassuolo',
 'fc schalke 04':'schalke 04', 'sevilla fc':'sevilla', 'sheffield united':'sheffield utd',
 'southampton fc':'southampton', 'spezia calcio':'spezia', 'rc strasbourg alsace':'strasbourg',
 'vfb stuttgart':'stuttgart', 'torino fc':'torino', 'tottenham hotspur':'tottenham',
 'udinese calcio':'udinese', '1 fc union berlin':'union berlin', 'valencia cf':'valencia', 'real valladolid cf':'valladolid',
 'villarreal cf':'villarreal', 'sv werder bremen':'werder bremen', 'west bromwich albion':'west brom',
 'west ham united':'west ham', 'vfl wolfsburg':'wolfsburg', 'wolverhampton wanderers':'wolves'}

tm_df.replace(replace_dict, inplace=True)

In [24]:
# Use fuzzymatcher package to join the two datasets based on player name and team name
final_df = fuzzymatcher.fuzzy_left_join(combined_df, tm_df, left_on=['Player', 'Team'], right_on=['Player', 'Team'])
# Drop values that have a matching below 0.3
final_df.drop(final_df[final_df.best_match_score <= 0.3].index, inplace=True)

In [25]:
final_df.columns

Index(['best_match_score', '__id_left', '__id_right', 'Player_left',
       'Position', 'Team_left', 'Comp', 'Age', 'MP', 'Min', 'Goals', 'Ast',
       'G+A', 'xG', 'xA', 'SoT%', 'npxG/Sh', 'G-xG', 'Pass Cmp%',
       'Prog Passes', 'Through Balls', 'Passes Under Pressure',
       'Switch Passes', 'Crosses', 'Shot Creating Actions', 'Dribbles to Shot',
       'Def Action to Shot', 'Tackles', 'Press', 'Shots Blocked',
       'Interceptions', 'Clearances', 'Errors', 'Dribbles Successful',
       'Dribble Distance', 'Mistakes', 'Ball Recoveries', 'Air Duels Won%',
       'Player_right', 'Values', 'Team_right'],
      dtype='object')

In [26]:
# Clean some of the labels
final_df.drop(labels=['best_match_score', '__id_left', '__id_right', 'Player_right', 'Team_right'], axis=1, inplace=True)
final_df.reset_index(drop=True, inplace=True)
final_df.rename(columns={"Player_left": "Player", "Team_left": "Team"}, inplace=True)

In [29]:
final_df

,Player,Position,Team,Comp,Age,MP,Min,Goals,Ast,G+A,...,Shots Blocked,Interceptions,Clearances,Errors,Dribbles Successful,Dribble Distance,Mistakes,Ball Recoveries,Air Duels Won%,Values
0,Bruno Ecuele Manga,DF,dijon,fr Ligue 1,32,38,3420,0.05,0.00,0.05,...,1.00,2.16,5.50,0.05,0.26,89.5,0.16,9.16,69.7,1320000.0
1,Pierre Hojbjerg,MF,tottenham,eng Premier League,24,38,3420,0.05,0.11,0.16,...,0.26,1.29,2.16,0.00,0.66,114.1,0.74,11.30,56.5,38500000.0
2,James Ward-Prowse,MF,southampton,eng Premier League,25,38,3420,0.21,0.18,0.39,...,0.13,1.16,2.00,0.00,0.39,91.4,0.42,12.70,43.4,33000000.0
3,Dylan Bronn,DF,metz,fr Ligue 1,25,38,3412,0.05,0.00,0.05,...,0.42,1.00,4.30,0.08,0.42,117.3,0.13,8.42,57.1,6600000.0
4,Ashley Westwood,MF,burnley,eng Premier League,30,38,3410,0.08,0.08,0.16,...,0.26,1.27,1.29,0.00,0.32,64.6,0.50,11.10,31.1,5500000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1861,Sergio Gonzalez,MFDF,cadiz,es La Liga,23,1,90,0.00,0.00,0.00,...,0.00,2.00,2.00,0.00,0.00,37.0,2.00,13.00,40.0,550000.0
1862,Kayne Ramsey,DF,southampton,eng Premier League,19,1,90,0.00,0.00,0.00,...,1.00,1.00,4.00,0.00,0.00,12.0,0.00,6.00,50.0,275000.0
1863,Yoann Etienne,DF,lorient,fr Ligue 1,23,1,90,0.00,0.00,0.00,...,0.00,2.00,0.00,0.00,0.00,48.0,2.00,10.00,25.0,715000.0
1864,Mark Kosznovszky,MF,parma,it Serie A,18,1,90,0.00,0.00,0.00,...,0.00,3.00,0.00,0.00,0.00,69.0,1.00,8.00,46.4,110000.0


In [30]:
# Save final dataset as csv for modeling
final_df.to_csv("Final_Dataset.csv")